<a href="https://colab.research.google.com/github/shunsukemlab/espnet_onnx/blob/master/demo/simple_asr_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# espnet_onnx demonstration

This notebook provides a simple demonstration of how to export your trained model and use it for inference.

see also:
- ESPnet: https://github.com/espnet/espnet
- espnet_onnx: https://github.com/Masao-Someki/espnet_onnx

Author: [Masao Someki](https://github.com/Masao-Someki)


## Table of Contents

- Install Dependency
- Export your model
- Inference with onnx

# Install Dependency
To run this demo, you need to install the following packages.
- espnet_onnx
- torch >= 1.11.0 (already installed in Colab)
- espnet
- espnet_model_zoo
- onnx

`torch`, `espnet`, `espnet_model_zoo`, `onnx` is required to run the exportation demo.

pythonとtorchをインストールしなおす

In [ ]:
!sudo apt install python3.8
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!python --version
!apt-get install python3.8-distutils
!apt-get install python3-pip

In [ ]:
!wget https://download.pytorch.org/whl/cpu/torch-2.0.1%2Bcpu-cp38-cp38-linux_x86_64.whl
!pip install torch-2.0.1%2Bcpu-cp38-cp38-linux_x86_64.whl

In [ ]:
!pip install -U espnet_onnx espnet espnet_model_zoo onnx --no-cache-dir

# in this demo, we need to update scipy to avoid an error
!pip install -U scipy numpy==1.23.5 pyworld==0.3.2

And we need additional dependency `onnxruntime-gpu` to run inference on the GPU.

In [ ]:
!pip install onnxruntime==1.14.1

In [ ]:
!wget https://github.com/espnet/espnet_onnx/releases/download/custom_ort_v1.14.1.espnet/onnxruntime-1.14.1.espnet-cp38-cp38-linux_x86_64.whl

In [ ]:
!mv onnxruntime-1.14.1.espnet-cp38-cp38-linux_x86_64.whl onnxruntime-1.14.1-cp38-cp38-linux_x86_64.whl

In [ ]:
!pip install ./onnxruntime-1.14.1-cp38-cp38-linux_x86_64.whl

In [ ]:
!pip install torch==2.0.1 --upgrade

In [ ]:
!pip install torchaudio==2.0.1

# Export your model

## Export model from espnet_model_zoo

The easiest way to export a model is to use `espnet_model_zoo`. You can download, unpack, and export the pretrained models with `export_from_pretrained` method.
`espnet_onnx` will save the onnx models into cache directory, which is `${HOME}/.cache/espnet_onnx` in default.

In [ ]:
!wget https://raw.githubusercontent.com/espnet/espnet_onnx/master/espnet_onnx/export/convert_map.yml -O /usr/local/lib/python3.10/dist-packages/espnet_onnx/export/convert_map.yml


In [ ]:
# export the model.
from espnet_onnx.export import ASRModelExport

tag_name = 'reazon-research/reazonspeech-espnet-v2'

m = ASRModelExport()
m.set_export_config(
    max_seq_len=5000,
)
m.export_from_pretrained(
    tag_name,  optimize=True,
  quantize=False
)

# Inference with onnxruntime
Now, let's use the exported models for inference.
Please enable the GPU resource to run the following codes.

In [ ]:
# please provide the tag_name to specify exported model.
tag_name = 'reazon-research/reazonspeech-espnet-v2'
export_dir = f'/root/.cache/espnet_onnx/{tag_name}'
# upload wav file and let's inference!
import librosa
from google.colab import files

wav_file = files.upload()
y, sr = librosa.load(list(wav_file.keys())[0], sr=16000)

# Use the exported onnx file to inference.
from espnet_onnx import Speech2Text

speech2text = Speech2Text(model_dir=export_dir, providers=['CPUExecutionProvider'])
nbest = speech2text(y)
print(nbest[0][0])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /root/.cache/espnet_onnx/reazon-research/reazonspeech-espnet-v2/

In [ ]:
import torch
print(torch.__version__)